<a href="https://colab.research.google.com/github/grkmkcgl/globalai_bootcamp_project/blob/main/GlobalAI_Project_DataPrep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bu kısımları kendi bilgisayarlarımızdan spyder kullanarak yaptık. Spyder total çıktısı en aşağıda yer alıyor

**Kütüphanelerin import edilmesi**

In [ ]:
import librosa.feature
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import librosa.display
import cv2 as cv2

**Verilerin colab'e indirilmesi**

In [ ]:
!wget https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz -O urban8k.tgz
!tar -xzf urban8k.tgz
!rm urban8k.tgz

--2022-09-24 12:12:14--  https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6023741708 (5.6G) [application/octet-stream]
Saving to: ‘urban8k.tgz’

urban8k.tgz         100%[===================>]   5.61G  20.4MB/s    in 4m 5s   

2022-09-24 12:16:21 (23.4 MB/s) - ‘urban8k.tgz’ saved [6023741708/6023741708]



**Spectogramların oluşturulması (Bu kısmı spyder üzerinden kendi bilgisayarlarımızdan yaptık çünkü veri dosyasını upload etmesi çok uzun sürüyordu)**

In [ ]:
def create_spectrogram(y):
    spec = librosa.feature.melspectrogram(y=y)
    spec_conv = librosa.amplitude_to_db(spec, ref=np.max)
    return spec_conv

directory = 'UrbanSound8K/audio'


# Create spectograms and save figures in folders

for foldername in os.listdir(directory):
    folder = os.path.join(directory, foldername)
    if os.path.isdir(folder):
        audiofolder = folder[19:]
        print(audiofolder)
        os.mkdir(f"CreatedSpectrograms/{audiofolder}")
        for audio in os.listdir(folder):
            file = os.path.join(folder, audio)
            if os.path.isfile(file):
                if not file.endswith(".DS_Store"):
                    signal, sr = librosa.load(file)
                    spectrogram = create_spectrogram(signal)
                    np.shape(spectrogram)
                    hop_length = 512
                    n_fft = 2048
                    plt.figure(figsize=(8, 7))
                    librosa.display.specshow(spectrogram, sr=sr, cmap="magma", hop_length=hop_length)
                    plt.savefig(f"CreatedSpectrograms/{audiofolder}/{audio[:-4]}.png")
                    plt.close()

**Spectogramların resim listesine dönüştürülmesi, sınıfıyla birlikte kaydedilmesi ve veri setine dönüştürülmesi**

In [ ]:
import numpy as np
import cv2
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Gereksiz kolonları silme
directory = "spectrograms"
data = pd.read_csv("UrbanSound8K.csv")
data.drop(['fsID', 'start', 'end', 'salience', 'fold', 'class', 'slice_file_name'], inplace=True, axis=1)
data.sort_values(by='classID', inplace = True)
data.reset_index(inplace = True)
data.drop(['index'], inplace=True, axis=1)

# Resimler ve etiketler için boş liste oluşturma
list_of_specs_list = []
row = 0

for foldername in os.listdir(directory):
    folder = os.path.join(directory, foldername)
    print("folder", foldername, "analyzing has started")
    if os.path.isdir(folder):
        for spec in os.listdir(folder):
            image = os.path.join(folder, spec)
            # resmi okuma ve grayscale çevirme
            image = cv2.imread(image)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            # resmi 32,32,1 boyutlandırma
            image = cv2.resize(image, (32, 32))
            image = np.array(image, dtype=np.float64)
            image = np.expand_dims(image, -1)
            # normalize etme
            image /= 255
            # class'ını alıp kaydetme
            classID = data.iloc[row:row+1, 0:1].values
            classID = np.array(classID, dtype='uint8')
            row +=  1
            list_of_specs_list.append([image,classID])

print("DONE CONVERTING TO LIST")

# veriyi karıştırma
random.shuffle(list_of_specs_list)

**Veri setlerinin csv dosyasına dönüştürülmesi**

In [ ]:
images = []
classes = []

for image, classID in list_of_specs_list:
    images.append(image)
    classes.append(classID)

# np arraye dönüştürme
images = np.array(images).reshape(-1, 32, 32, 1)


# train test split oluşturma
X_train, X_temporary, y_train, y_temporary = train_test_split(images, classes, 
                                                    test_size=0.2,
                                                    random_state=0)

X_val, X_test, y_val, y_test = train_test_split(X_temporary, y_temporary, train_size=0.5)


# dataları csv ye kaydetme
shaped_arr = np.array(X_train).reshape(-1, 32)
shaped_arr = pd.DataFrame(shaped_arr)
shaped_arr.to_csv("X_train.csv", index=False, compression={'method': 'gzip', 'compresslevel': 1, 'mtime': 1})

shaped_arr = np.array(X_test).reshape(-1, 32)
shaped_arr = pd.DataFrame(shaped_arr)
shaped_arr.to_csv("X_test.csv", index=False, compression={'method': 'gzip', 'compresslevel': 1, 'mtime': 1})

shaped_arr = np.array(X_val).reshape(-1, 32)
shaped_arr = pd.DataFrame(shaped_arr)
shaped_arr.to_csv("X_val.csv", index=False, compression={'method': 'gzip', 'compresslevel': 1, 'mtime': 1})

shaped_arr = np.array(y_train).reshape(-1, 32)
shaped_arr = pd.DataFrame(shaped_arr)
shaped_arr.to_csv("y_train.csv", index=False, compression={'method': 'gzip', 'compresslevel': 1, 'mtime': 1})

shaped_arr = np.array(y_test).reshape(-1, 32)
shaped_arr = pd.DataFrame(shaped_arr)
shaped_arr.to_csv("y_test.csv", index=False, compression={'method': 'gzip', 'compresslevel': 1, 'mtime': 1})

shaped_arr = np.array(y_val).reshape(-1, 32)
shaped_arr = pd.DataFrame(shaped_arr)
shaped_arr.to_csv("y_val.csv", index=False, compression={'method': 'gzip', 'compresslevel': 1, 'mtime': 1})

OUTPUT:

folder 0 analyzing has started

folder 1 analyzing has started

folder 2 analyzing has started

folder 3 analyzing has started

folder 4 analyzing has started

folder 5 analyzing has started

folder 6 analyzing has started

folder 7 analyzing has started

folder 8 analyzing has started

folder 9 analyzing has started

DONE CONVERTING TO NESTED LIST